In [35]:
import math

def eval_conductor_section(fault_current, kf, trip_time):        
    conductor_section_kcmil = fault_current*kf*math.sqrt(trip_time)
    conductor_section_mm = conductor_section_kcmil/1.974
    diameter = (2*math.sqrt(conductor_section_mm/math.pi))/1000 # Diameter should be returned in meters
    return diameter

def calc_max_values(trip_time, ground_ro, gravel_ro, gravel_depth):
    max_current = 0.157/math.sqrt(trip_time)
    correction_factor = 1 - (0.09*(1-ground_ro/gravel_ro
                                   )/(2*gravel_depth+0.09))

    max_touch_voltage = (1000+1.5*correction_factor*gravel_ro)*max_current
    max_step_voltage = (1000+6*correction_factor*gravel_ro)*max_current
    print(correction_factor)
    return max_touch_voltage, max_step_voltage

def eval_grid_resistance(width, height, depth, spacement, ground_ro):
    horizontal_conductors = (height/spacement)+1
    vertical_conductors = (width/spacement) + 1
    total_length = horizontal_conductors*width + vertical_conductors*height
    area = width*height
    equivalent_resistance = ground_ro*(
        (1/total_length)+(1/math.sqrt(20*area))*(
        1 + (1/(1+depth*math.sqrt(20/area)))))
    return equivalent_resistance

def eval_gpr(maximum_current, grid_resistance):
    gpr = maximum_current*grid_resistance
    return gpr

In [37]:
grid_r = eval_grid_resistance(70, 70, 0.5, 7, 400)
print(eval_gpr(1908, grid_r))
print(grid_r)

5296.0235087671645
2.7756936628758724
